In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_train = pd.read_csv('/kaggle/input/mobile-price-classification/train.csv', encoding='utf-8')
df_train.head()

In [ ]:
df_test = pd.read_csv('/kaggle/input/mobile-price-classification/test.csv', encoding='utf-8')
df_test.head()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
print(df_train.duplicated().sum())
print(df_test.duplicated().sum())

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_train.plot(kind='box', subplots=True, figsize=(20,20), layout=(5,5))
plt.show()

In [ ]:
df_train['three_g'].value_counts()

In [ ]:
df_test.plot(kind='box', subplots=True, figsize=(20,20), layout=(5,5))
plt.show()

In [ ]:
plt.figure(figsize=(20,7))
sns.heatmap(df_train.corr() , annot = True, cmap = "Blues")

## Modeling

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

sc = StandardScaler()
x = df_train.drop(['price_range'] , axis = 1).values
y =df_train['price_range'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=42)
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [ ]:
print(x_train.shape)
print(x_test.shape)

### Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(x_train, y_train)

print(logreg.score(x_train, y_train))
print(logreg.score(x_test, y_test))

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth= 5, max_features= 17)
dt.fit(x_train, y_train)
print (dt.score(x_train, y_train))
print (dt.score(x_test, y_test))

### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators= 5, max_depth=5 ,max_features= 17)
rf.fit(x_train , y_train)
print (rf.score(x_train , y_train))
print (rf.score(x_test , y_test))

### features importance

In [ ]:
from sklearn import tree
fig = plt.figure(figsize= (15,12))
tree.plot_tree(dt, filled= True)

In [ ]:
i = df_train.drop(['price_range'] , axis = 1)
j =df_train['price_range']

In [ ]:
i.columns

In [ ]:
def f_importances(coef, names, top=-1):
    imp = coef
    imp, names = zip(*sorted(list(zip(imp, names))))

    # Show all features
    if top == -1:
        top = len(names)

    plt.barh(range(top), imp[::-1][0:top], align='center')
    plt.yticks(range(top), names[::-1][0:top])
    plt.title('feature importances')
    plt.show()

# whatever your features are called
features_names = i.columns

# Specify your top n features you want to visualize.
# You can also discard the abs() function 
# if you are interested in negative contribution of features
f_importances(abs(dt.feature_importances_), features_names, top=6)


In [ ]:
(pd.Series(dt.feature_importances_, index=i.columns).plot(kind='barh'))  

## XGBOOST

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators = 15, max_depth = 3)
xgb.fit(x_train, y_train)
print(xgb.score(x_train, y_train))
print(xgb.score(x_test, y_test))

## ADABOOST


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()
ada.fit(x_train, y_train)
print(ada.score(x_train, y_train))
print(ada.score(x_test, y_test))

In [ ]:
ada.get_params()

 ## Voting

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
clf1 = LogisticRegression()
clf2 = AdaBoostClassifier()
clf3 = XGBClassifier(n_estimators = 15, max_depth = 3)
clf4 = RandomForestClassifier(n_estimators= 5, max_depth=5 ,max_features= 17)

In [ ]:
v_clf = VotingClassifier(estimators=[('logreg', clf1), ('ada', clf2), ('xgb', clf3), ('RF', clf4)], voting='soft')
v_clf.fit(x_train, y_train)

In [ ]:
#soft voting
print(v_clf.score(x_train, y_train))
print(v_clf.score(x_test, y_test))

In [ ]:
v_clf = VotingClassifier(estimators=[('logreg', clf1), ('ada', clf2), ('xgb', clf3), ('RF', clf4)], voting='hard')
v_clf.fit(x_train, y_train)

In [ ]:
#hard voting
print(v_clf.score(x_train, y_train))
print(v_clf.score(x_test, y_test))

### Select best features

### Evaluation

In [ ]:
y_pred = xgb.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
con = confusion_matrix(y_test, y_pred)
con

In [ ]:
#pip install mlxtend

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
plot_confusion_matrix(con)

In [ ]:
y_test.shape

In [ ]:
print(classification_report(y_test, xgb.predict(x_test)))

In [ ]:
print(classification_report(y_test, rf.predict(x_test)))

In [ ]:
print(classification_report(y_test, dt.predict(x_test)))

### Cross Validation

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scoring = 'accuracy'
score = cross_val_score(rf, x, y, cv = kfold, scoring = scoring)
print(score)

In [ ]:
round(np.mean(score)*100, 2)

### Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth': [2,3,4,5], 'max_features': [7,8,9], 'n_estimators': [1,7,8,9]}
param_grid

In [ ]:
clf = RandomForestClassifier()
grid = GridSearchCV(estimator=clf, param_grid = param_grid, cv=5)
grid_result = grid.fit(x_train, y_train)
grid_result

In [ ]:
print('Best: %f using %s' %(grid_result.best_score_, grid_result.best_params_))

In [ ]:
model = grid_result.best_estimator_
model

In [ ]:
con = confusion_matrix()
con

In [ ]:
#confusion matrix
#classification report
#cross validations
#grid search
#plot_roc_curve